In [3]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import time
import pandas as pd
import numpy as np
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [57]:
# Scrape daily gross for a specific date to test

url = 'https://www.boxofficemojo.com/date/2021-08-08/'
page = requests.get(url)
soup = bsoup(page.text, 'lxml')

all_divs = soup.main.find_all("div", id="table")
all_tables = all_divs[0].find_all("table")
all_trs = all_tables[0].find_all("tr")

for i in range(1, len(all_trs)):
    print(f"Entry row {i} = ")
    
    all_tds = all_trs[i].find_all("td")
    a_href = all_tds[2].find('a',href=True)
    entry = []
    
    for j in range(0, len(all_tds)):
        entry.append(all_tds[j].text)
    
    # go to individual movie page to get genres
    a_href = 'http://boxofficemojo.com' + a_href['href']
    movie_page = requests.get(a_href)
    movie_soup = bsoup(movie_page.text, 'lxml')
    
    table_div = movie_soup.find('div', {'class': 'mojo-summary-values'})
    sub_divs = table_div.find_all('div')

    genres = []
    for s in sub_divs:
        if re.search('^Genres',s.text):
            genres = s.text
            genres = re.sub('Genres', '', genres)
            genres = re.sub('\s+', ' ', genres)
    entry.append(genres.split())
        
    print(entry)

Entry row 1 = 
['1', '1', 'The Suicide Squad', '$5,940,000', '-27.6%', '-', '-', '-', '$26,205,415', '3', 'Warner Bros.\n\n', 'false', 'true', ['Action', 'Adventure', 'Comedy', 'Sci-Fi']]
Entry row 2 = 
['2', '2', 'Jungle Cruise', '$4,616,034', '-30.8%', '-50%', '4,310', '$1,071', '$65,443,044', '10', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false', ['Action', 'Adventure', 'Comedy', 'Fantasy']]
Entry row 3 = 
['3', '3', 'Old', '$1,213,725', '-28.3%', '-41.4%', '3,138', '$386', '$38,498,750', '17', 'Universal Pictures\n\n', 'false', 'false', ['Drama', 'Mystery', 'Thriller']]
Entry row 4 = 
['4', '4', 'Black Widow', '$1,082,915', '-35.1%', '-42.3%', '3,100', '$349', '$174,280,036', '31', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false', ['Action', 'Adventure', 'Sci-Fi']]
Entry row 5 = 
['5', '5', 'Stillwater', '$823,955', '-32.9%', '-41.1%', '2,611', '$315', '$9,968,760', '10', 'Focus Features\n\n', 'false', 'false', ['Crime', 'Drama', 'Thriller']]
Entry row 6 = 
[

In [69]:
# Build date for URL and make requests for each day from 2015-2019

month_and_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,29)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

leap_years = [2008, 2016, 2020]
leap_year_month_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,30)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

all_data = []

for year in range(2015, 2020):
    # build url for each year-month-day and scrape table data
    url = 'https://www.boxofficemojo.com/date/'
    
    if year in leap_years:
        m_d = leap_year_month_days
    else:
        m_d = month_and_days
        
    y = str(year)
    
    for month in m_d:
        m = '-' + month
        for day in m_d[month]:
            if len(str(day)) == 1:
                d = '-' + '0' + str(day)
            else:
                d = '-' + str(day)
            
            date = y + m + d
            url_curr = url + date
            
            #get data
            page = requests.get(url_curr)
            soup = bsoup(page.text, 'lxml')
            
            all_divs = soup.main.find_all("div", id="table")
            
            try:
                all_tables = all_divs[0].find_all("table")
            except:
                print(date)
            
            try:
                all_trs = all_tables[0].find_all("tr")
            except:
                print(date)
            
            for i in range(1, len(all_trs)):

                all_tds = all_trs[i].find_all("td")
                a_href = all_tds[2].find('a',href=True)
                a_href = 'http://boxofficemojo.com' + a_href['href']
                entry = []

                for j in range(0, len(all_tds)):
                    entry.append(all_tds[j].text)       
                
                entry = entry[0:11]
                entry.append(a_href)
                entry.append(date)

            
                all_data.append(entry)
    print(f'{year} processed...')

2015 processed...
2016 processed...
2017 processed...
2018 processed...
2019 processed...


In [70]:
len(all_data)

95953

In [83]:
column_names = ['TD', 'YD', 'Movie', 'Daily', '%YD', '%LW', 'Theaters', 'Avg', 'To_Date', 
                'Days', 'Distributor', 'href','Date']
df = pd.DataFrame(all_data, columns=column_names)
df.to_pickle('boxofficemojo-daily_gross.pkl')
df

NameError: name 'all_data' is not defined

In [ ]:
df = pd.read_pickle('boxofficemojo-daily_gross.pkl')

In [69]:
titles_unique = list(set(df['Movie']))
len(titles_unique)

2353

In [78]:
# Go to each movie page to scrape genre data and add to dataframe
genre_dict = {}

for m in titles_unique:
    h = list(df.loc[df['Movie'] == m, 'href'])[0]
    movie_page = requests.get(h)
    movie_soup = bsoup(movie_page.text, 'lxml')

    table_div = movie_soup.find('div', {'class': 'mojo-summary-values'})
    sub_divs = table_div.find_all('div')

    for s in sub_divs:
        if re.search('^Genres',s.text):
            genres = s.text
            genres = re.sub('Genres', '', genres)
            genres = re.sub('\s+', ' ', genres)
    genre_dict[m] = genres.split()

genre_dict

{'Valerian and the City of a Thousand Planets': ['Action',
  'Adventure',
  'Fantasy',
  'Sci-Fi'],
 'Wish Upon': ['Drama', 'Fantasy', 'Horror', 'Mystery', 'Thriller'],
 'The Last Black Man in San Francisco': ['Drama'],
 'A Man Vanishes2012 Re-release': ['Documentary'],
 'The Sound of Music2018 Re-release': ['Biography',
  'Drama',
  'Family',
  'Musical',
  'Romance'],
 'Indignation': ['Drama', 'Romance'],
 'Pokémon the Movie: The Power of Us': ['Action',
  'Adventure',
  'Animation',
  'Family',
  'Fantasy',
  'Sci-Fi'],
 'Hamlet2019 Re-release': ['Drama'],
 'Quai des Orfèvres2018 Re-release': ['Crime', 'Drama'],
 'My Journey Through French Cinema': ['Documentary'],
 "Don't Breathe": ['Crime', 'Horror', 'Thriller'],
 'Christ Stopped at Eboli': ['Drama'],
 'City of Ghosts': ['Documentary', 'War'],
 'Legend': ['Biography', 'Crime', 'Drama', 'Thriller'],
 'Strangerland': ['Drama', 'Mystery', 'Thriller'],
 'Wonder Park': ['Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy'],
 "India'

In [79]:
# Check
len(genre_dict)

2353

In [58]:
# Test to unpack genres and add to dataframe
genres = genre_dict['http://boxofficemojo.com/release/rl654673409/?ref_=bo_da_table_9']
genre_str = ''
for g in genres:
    genre_str += g + '|'
df.loc[df['href']=='http://boxofficemojo.com/release/rl654673409/?ref_=bo_da_table_9', 'genre'] = genre_str
df.loc[df['href']== 'http://boxofficemojo.com/release/rl654673409/?ref_=bo_da_table_9']

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date,genre
81826,9,10,Hellboy,1563100,16.3,63.2,3303.0,473.0,18703695.0,9.0,Lionsgate,http://boxofficemojo.com/release/rl654673409/?...,2019-04-20,Action|Adventure|Fantasy|Horror|Sci-Fi|


In [87]:
# For each href in the dictionary, unpack genres into string and add to genre column for that film
for m in genre_dict:
    genres = genre_dict[m]
    genre_str = ''
    for g in genres:
        genre_str += g + '|'
    df.loc[df['Movie'] == m,'genre'] = genre_str

df.genre.sample(50)

92897                                       Drama|Romance|
25096                           Action|Drama|Thriller|War|
47692                                Crime|Drama|Thriller|
76274                       Comedy|Family|Fantasy|Musical|
16032                              Drama|Mystery|Thriller|
83600                  Adventure|Animation|Fantasy|Sci-Fi|
19093                                   Drama|History|War|
70403    Adventure|Animation|Comedy|Family|Fantasy|Musi...
93245                                  Comedy|Crime|Drama|
6603     Adventure|Animation|Comedy|Family|Fantasy|Sci-Fi|
15955                                       Drama|Romance|
27231                       Action|Horror|Sci-Fi|Thriller|
57589                     Adventure|Comedy|Fantasy|Sci-Fi|
19977                       Biography|Drama|Music|Romance|
12209           Action|Adventure|Biography|Drama|Thriller|
35951                                        Drama|Sci-Fi|
21209                          Biography|Comedy|Drama|Wa

In [88]:
unique_genre =  df.genre.str.split('|').sum()
unique_genre = set(unique_genre)
unique_genre

for g in unique_genre:
    df[g] = df.genre.map( lambda x: g in x.split('|') )
    
df.sample(50)

22319     Adventure|Animation|Comedy|Crime|Family|Mystery|
10623                      Action|Mystery|Sci-Fi|Thriller|
20639                                       Comedy|Family|
1242                                                Drama|
63546                             Action|Adventure|Sci-Fi|
81493                              Comedy|Fantasy|Romance|
67386                                Comedy|Drama|Romance|
75706                             Action|Adventure|Sci-Fi|
67098                              Biography|Comedy|Drama|
77656                         Adventure|Animation|Musical|
88060                                        Comedy|Drama|
31915                              Biography|Comedy|Crime|
93185                                               Drama|
18298                Action|Comedy|Fantasy|Horror|Romance|
37642                               Action|Crime|Thriller|
6897                                Drama|Sci-Fi|Thriller|
25168                            Adventure|Family|Fantas

In [94]:
df.genre.isnull().sum()

0

In [90]:
# Replace non-numeric values for numeric data columns
df['Daily'] = df['Daily'].str.replace('[^0-9]', '')
df['To_Date'] = df['To_Date'].str.replace('[^0-9]', '')
df['%YD'] = df['%YD'].str.replace('+', '')
df['%YD'] = df['%YD'].str.replace('-', '')
df['%YD'] = df['%YD'].str.replace('%', '')
df['%YD'] = df['%YD'].str.replace(',', '')
df['%YD'] = df['%YD'].str.replace('<0.1', '0')
df['%LW'] = df['%LW'].str.replace('+', '')
df['%LW'] = df['%LW'].str.replace('-', '')
df['%LW'] = df['%LW'].str.replace('%', '')
df['%LW'] = df['%LW'].str.replace(',', '')
df['%LW'] = df['%LW'].str.replace('<0.1', '0')
df['Theaters'] = df['Theaters'].str.replace('[^0-9]', '')
df['Avg'] = df['Avg'].str.replace('[^0-9]', '')
df['Days'] = df['Days'].str.replace('[^0-9]', '')
df['Distributor'] = df['Distributor'].str.replace('\\n', '')
df.head()

AttributeError: Can only use .str accessor with string values!

In [74]:
# Update numeric data types
df['Daily'] = pd.to_numeric(df['Daily'])
df['%YD'] = pd.to_numeric(df['%YD'])
df['%LW'] = pd.to_numeric(df['%LW'])
df['Theaters'] = pd.to_numeric(df['Theaters'])
df['Avg'] = pd.to_numeric(df['Avg'])
df['To_Date'] = pd.to_numeric(df['To_Date'])
df['Days'] = pd.to_numeric(df['Days'])
df.dtypes

TD              object
YD              object
Movie           object
Daily            int64
%YD            float64
%LW            float64
Theaters       float64
Avg            float64
To_Date        float64
Days           float64
Distributor     object
href            object
Date            object
dtype: object

In [75]:
df['Date'] = pd.to_datetime(df['Date'])

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date
32043,27,27,Kubo and the Two Strings,49048,17.2,134.1,176.0,278.0,47612370.0,73.0,Focus Features,http://boxofficemojo.com/release/rl3881600513/...,2016-10-30
84267,3,3,Rocketman,7273218,21.6,NaN,3610.0,2014.0,25725722.0,3.0,Paramount Pictures,http://boxofficemojo.com/release/rl4261709313/...,2019-06-02
85386,35,30,Wild Rose,8138,60.0,NaN,4.0,2034.0,52938.0,3.0,Neon,http://boxofficemojo.com/release/rl2349303297/...,2019-06-23
11571,40,51,The Cokeville Miracle,2759,370.0,75.6,19.0,145.0,1325176.0,102.0,Excel Entertainment,http://boxofficemojo.com/release/rl1635550721/...,2015-09-14
64240,37,35,On Chesil Beach,4483,32.9,51.3,24.0,186.0,714482.0,38.0,Bleecker Street Media,http://boxofficemojo.com/release/rl3976365569/...,2018-06-24
62576,49,46,"Love, Simon",2674,21.8,0.8,32.0,83.0,40808933.0,71.0,Twentieth Century Fox,http://boxofficemojo.com/release/rl2768274945/...,2018-05-25
73760,47,45,The Sisters Brothers,746,35.2,57.0,5.0,149.0,3131465.0,76.0,Annapurna Pictures,http://boxofficemojo.com/release/rl1376224769/...,2018-12-05
45916,1,1,Dunkirk,7530839,32.7,43.1,3748.0,2009.0,101317350.0,10.0,Warner Bros.,http://boxofficemojo.com/release/rl4118644225/...,2017-07-30
2818,2,-,The Lazarus Effect,3786762,NaN,NaN,2666.0,1420.0,3786762.0,1.0,Relativity Media,http://boxofficemojo.com/release/rl3680470529/...,2015-02-27
25967,44,42,Saturday's Warrior,656,38.6,35.8,7.0,93.0,670989.0,110.0,Purdie Distribution,http://boxofficemojo.com/release/rl4252993025/...,2016-07-19


In [66]:
df.sample(20)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,...,Action,Romance,Biography,Sport,Sci-Fi,Drama,Fantasy,Animation,Mystery,Adventure
10323,39,41,The Young and Prodigious T.S. Spivet,220,13.7,86.0,9.0,24.0,76878.0,19.0,...,False,False,False,False,False,False,False,False,False,False
24744,3,-,The Shallows,6880944,NaN,NaN,2962.0,2323.0,6880944.0,1.0,...,False,False,False,False,False,False,False,False,False,False
67439,54,54,"Don't Worry, He Won't Get Far on Foot",1812,21.6,72.5,19.0,95.0,1420587.0,45.0,...,False,False,True,False,False,True,False,False,False,False
14149,38,34,Ladrones,1777,80.1,76.6,39.0,45.0,3034330.0,32.0,...,False,False,False,False,False,False,False,False,False,False
91548,9,8,It Chapter Two,407616,54.6,3.6,2303.0,176.0,207454455.0,39.0,...,False,False,False,False,False,False,False,False,False,False
82039,13,15,Kalank,187336,71.2,NaN,320.0,585.0,2083538.0,7.0,...,False,False,False,False,False,False,False,False,False,False
87289,14,14,Avengers: Endgame,154435,18.3,49.7,580.0,266.0,856861364.0,96.0,...,False,False,False,False,False,False,False,False,False,False
7013,2,2,Pitch Perfect 2,6095050,32.7,44.9,3660.0,1665.0,143387880.0,16.0,...,False,False,False,False,False,False,False,False,False,False
52556,39,38,Battle of the Sexes,9835,36.7,56.8,46.0,213.0,12552907.0,59.0,...,False,False,False,False,False,False,False,False,False,False
65814,39,39,Black Panther,5292,9.9,250.7,154.0,34.0,699948596.0,160.0,...,False,False,False,False,False,False,False,False,False,False


In [91]:
df.dtypes

TD                     object
YD                     object
Movie                  object
Daily                   int64
%YD                   float64
%LW                   float64
Theaters              float64
Avg                   float64
To_Date               float64
Days                  float64
Distributor            object
href                   object
Date           datetime64[ns]
genre                  object
                         bool
Music                    bool
Short                    bool
Family                   bool
Horror                   bool
Crime                    bool
Documentary              bool
News                     bool
Musical                  bool
History                  bool
Western                  bool
War                      bool
Comedy                   bool
Thriller                 bool
Action                   bool
Romance                  bool
Biography                bool
Sport                    bool
Sci-Fi                   bool
Drama     

In [92]:
# Send data to pickle
df.to_pickle('clean-boxofficemojo-daily_gross.pkl')